<a href="https://colab.research.google.com/github/yvrjsharma/AI_workflows/blob/main/gemini_pro_and_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai
!pip install gradio==4.1.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.1

In [11]:
!pip install gradio_multimodalchatbot -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00


In [13]:
import os

import google.generativeai as genai
import gradio as gr
import PIL.Image
from gradio.data_classes import FileData
from gradio_multimodalchatbot import MultimodalChatbot

In [3]:
# For better security practices, retrieve sensitive information like API keys from environment variables.
# Set an environment variable
os.environ["GOOGLE_API_KEY"] = "enter-your-google-gemini-API-key-here"

# Fetch an environment variable.
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
# Verify if your api key is correct or not
!curl \
-H 'Content-Type: application/json' \
-d '{ "prompt": { "text": "Write a very short story about a magic backpack"} }' \
"https://generativelanguage.googleapis.com/v1beta3/models/text-bison-001:generateText?key=<enter-your-key-here>"

{
  "candidates": [
    {
      "output": "Once upon a time, there was a boy named Billy who found a magic backpack. The backpack could hold anything Billy wanted, and it would always be there when he needed it. Billy used the backpack to help people in need. He would give food to the hungry, clothes to the homeless, and even medical supplies to those in need. Billy's magic backpack made him a hero to everyone who knew him.",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_DEROGATORY",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_TOXICITY",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_VIOLENCE",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_SEXUAL",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_MEDICAL",
          "probability": "NEGLIGIBLE"
        },
        {
 

In [15]:
# Initialize genai models
model = genai.GenerativeModel("gemini-pro")
modelvis = genai.GenerativeModel("gemini-pro-vision")


def gemini(input, file, chatbot=[]):
    """
    Function to handle gemini model and gemini vision model interactions.

    Parameters:
    input (str): The input text.
    file (File): An optional file object for image processing.
    chatbot (list): A list to keep track of chatbot interactions.

    Returns:
    tuple: Updated chatbot interaction list, an empty string, and None.
    """

    messages = []
    print(chatbot)

    # Process previous chatbot messages if present
    if len(chatbot) != 0:
        for user, bot in chatbot:
            user, bot = user.text, bot.text
            messages.extend(
                [{"role": "user", "parts": [user]}, {"role": "model", "parts": [bot]}]
            )
        messages.append({"role": "user", "parts": [input]})
    else:
        messages.append({"role": "user", "parts": [input]})

    try:
        # Process image if file is provided
        if file is not None:
            with PIL.Image.open(file.name) as img:
                message = [{"role": "user", "parts": [input, img]}]
                response = modelvis.generate_content(message)
                gemini_video_resp = response.text
                messages.append({"role": "model", "parts": [gemini_video_resp]})

                # Construct list of messages in the required format
                user_msg = {
                    "text": input,
                    "files": [{"file": FileData(path=file.name)}],
                }
                bot_msg = {"text": gemini_video_resp, "files": []}
                chatbot.append([user_msg, bot_msg])
        else:
            response = model.generate_content(messages)
            gemini_resp = response.text

            # Construct list of messages in the required format
            user_msg = {"text": input, "files": []}
            bot_msg = {"text": gemini_resp, "files": []}
            chatbot.append([user_msg, bot_msg])
    except Exception as e:
        # Handling exceptions and raising error to the modal
        print(f"An error occurred: {e}")
        raise gr.Error(e)

    return chatbot, "", None

In [16]:
# Define the Gradio Blocks interface
with gr.Blocks() as demo:
    # Add a centered header using HTML
    gr.HTML("<center><h1>Gemini-PRO & Gemini-PRO-Vision API</h1></center>")

    # Initialize the MultimodalChatbot component
    multi = MultimodalChatbot(value=[], height=800)

    with gr.Row():
        # Textbox for user input with increased scale for better visibility
        tb = gr.Textbox(scale=4)

        # Upload button for image files
        up = gr.UploadButton("Upload Image", file_types=["image"], scale=1)

    # Define the behavior on text submission
    tb.submit(gemini, [tb, up, multi], [multi, tb, up])

    # Define the behavior on image upload
    # Using chained then() calls to update the upload button's state
    up.upload(lambda: gr.UploadButton("Uploading Image..."), [], up).then(
        lambda: gr.UploadButton("Image Uploaded"), [], up
    ).then(lambda: gr.UploadButton("Upload Image"), [], up)

# Launch the demo with a queue to handle multiple users
demo.queue().launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:889: UserWarning: api_name lambda already exists, using lambda_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:889: UserWarning: api_name lambda already exists, using lambda_2
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ee733f7aff825581f2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[]
[(MultimodalMessage(text='hi', files=[]), MultimodalMessage(text='Hello! How can I help you today?', files=[]))]
[(MultimodalMessage(text='hi', files=[]), MultimodalMessage(text='Hello! How can I help you today?', files=[])), (MultimodalMessage(text='explain the image?', files=[FileMessage(file=FileData(path='/tmp/gradio/738002faf37e5e07cf5704777d0476aee29f5140/ref4.png', url='https://ee733f7aff825581f2.gradio.live/file=/tmp/gradio/738002faf37e5e07cf5704777d0476aee29f5140/ref4.png', size=None, orig_name=None, mime_type='image/png'), alt_text=None)]), MultimodalMessage(text='The image shows an ancient Assyrian relief carving. The carving depicts a king or other high-ranking official standing before a group of people. The king is holding a staff or scepter in one hand and a box or other object in the other hand. The people before him are standing in various poses and appear to be listening to the king or official. The carving is finely detailed and provides a glimpse into the art and 